In [0]:
pip install confluent-kafka snowflake-connector-python pandas


Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import json
import pandas as pd
import snowflake.connector
from confluent_kafka import Consumer

#kafka configuration
conf={
    'bootstrap.servers':'pkc-619z3.us-east1.gcp.confluent.cloud:9092',
    'security.protocol':'SASL_SSL',
    'sasl.mechanism':'PLAIN',
    'sasl.username':'GLY357Xwaq23BPEB73',
    'sasl.password':'qU7C7F2fs3ajcZTSSfb0IgdxSoaf9cLiOtH5QYRQnLV2OvtORfsssxRKaQPY0XeZnc8',
    'group.id':'my-python-consumer-group',
    'auto.offset.reset':'earliest'
}

#kafka topic
topic="streaming_data"

#create kafka consumer
consumer=Consumer(conf)
consumer.subscribe([topic])

#snowflake connection
conn = snowflake.connector.connect(
    user="user1234",
    password="user@12345",
    account="XMNHO234-WEESE234",
    warehouse="COMPUTE_WH",
    database="STREAMINGDATA",
    schema="PUBLIC"
)

cursor = conn.cursor()

#create table if not exists
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS streaming_data (
        id STRING,
        name STRING,
        age INT,
        city STRING,
        timestamp FLOAT
    )
""")

#consume messages and insert into snowflake
while True:
    msg=consumer.poll(1.0)
    if msg is None:
        continue
    if msg.error():
        print(f"Consumer error: {msg.error()}")
        continue

    #convert message into json
    record=json.loads(msg.value().decode("utf-8"))
    print(f"Received: {record}")

    #convert record to pandas dataframe
    df=pd.DataFrame([record])

    #insert into snowflake
    for _,row in df.iterrows():
        cursor.execute("""
                    INSERT INTO streaming_data (id,name,age,city,timestamp)
                    VALUES (%s,%s,%s,%s,%s)                       
                       """,(row["id"],row["name"],row["age"],row["city"],row["timestamp"]))
    conn.commit()

consumer.close()
cursor.close()
conn.close()




Received: {'id': 'fc45f720-b268-4ebf-8aac-b44cd890322b', 'name': 'Richard Jones', 'age': 37, 'city': 'Robertbury', 'timestamp': 1739682619.5391133}
Received: {'id': '52bc159c-1954-40fb-b7eb-198458cfb731', 'name': 'Michelle Carroll', 'age': 24, 'city': 'Lake Haroldborough', 'timestamp': 1739682623.1009533}
Received: {'id': 'a9c77c2b-89aa-469d-98f2-b524d8e5bfd1', 'name': 'Anthony Gregory', 'age': 30, 'city': 'Carsonborough', 'timestamp': 1739682627.8384287}
Received: {'id': 'e49be8a0-0475-4137-be88-513d1b4ec968', 'name': 'Carol Nichols DDS', 'age': 24, 'city': 'Port Kennethhaven', 'timestamp': 1739682631.3958786}
Received: {'id': '91f93f95-fff0-40e9-941e-f0142d7176ae', 'name': 'Linda Graves', 'age': 52, 'city': 'East Henry', 'timestamp': 1739682642.0735278}
Received: {'id': '82283677-e0e8-40d2-b351-80a95d0b22f6', 'name': 'Timothy Huff', 'age': 54, 'city': 'New Kimberlyfort', 'timestamp': 1739682643.2600858}
Received: {'id': 'cf8caad7-e48e-4536-a232-00d6672bce10', 'name': 'Lee Pratt', 'ag